# Часть 3: Анализ погрешностей измерений

В этом ноутбуке мы изучим, как погрешности измерений влияют на вычисляемые величины.

## Содержание
1. Теория: L1 и L2 нормы
2. Класс для анализа погрешностей
3. Демонстрация на простой функции
4. Задача 1: Плотность
5. Задача 2: Кинетическая энергия
6. Задача 3: Период маятника
7. Задача 4: Радиоактивный распад
8. Задача 5: Угол наклонной плоскости
9. Задача 6 (БОНУС): Объем цилиндра с коррелированными ошибками

---
## 📖 Теория: Распространение погрешностей

### Проблема

В эксперименте мы измеряем величины $x$ и $y$ с погрешностями $\delta x$ и $\delta y$. 

Затем вычисляем некоторую функцию $z = f(x, y)$.

**Вопрос:** Какая погрешность будет у $z$?

### Решение: Используем полный дифференциал

Из курса производных знаем:
$$dz = \frac{\partial f}{\partial x} \cdot dx + \frac{\partial f}{\partial y} \cdot dy$$

Для погрешностей:
$$\delta z \sim \left|\frac{\partial f}{\partial x}\right| \cdot \delta x + \left|\frac{\partial f}{\partial y}\right| \cdot \delta y$$

Но **как именно складывать погрешности**? Есть два подхода!

---

### Способ 1: L1-норма (Линейное сложение)

$$\delta z_{L1} = \left|\frac{\partial f}{\partial x}\right| \cdot \delta x + \left|\frac{\partial f}{\partial y}\right| \cdot \delta y$$

**Когда использовать:**
- Для **гарантированной оценки наихудшего случая**
- Предполагаем, что все погрешности складываются в самую неблагоприятную сторону
- Консервативная оценка (часто завышенная)

**Применение:**
- Инженерия и безопасность (нужна гарантия)
- Систематические погрешности
- Когда важно, чтобы результат не выходил за допустимые границы

**Геометрия:** В плоскости $(\delta x, \delta y)$ область ошибок — это **ромб** (манхэттенское расстояние)

---

### Способ 2: L2-норма (Квадратичное сложение)

$$\delta z_{L2} = \sqrt{\left(\frac{\partial f}{\partial x} \cdot \delta x\right)^2 + \left(\frac{\partial f}{\partial y} \cdot \delta y\right)^2}$$

**Когда использовать:**
- Для **реалистичной статистической оценки**
- Предполагаем, что погрешности **случайны и независимы**
- Оценка ближе к реальности на практике

**Применение:**
- Погрешности имеют случайную природу (шум измерений)
- Много независимых измерений
- Научные исследования

**Геометрия:** В плоскости $(\delta x, \delta y)$ область ошибок — это **эллипс** (евклидово расстояние)

---

### Сравнение L1 и L2

**Всегда выполняется:**
$$\delta z_{L2} \leq \delta z_{L1}$$

L2-оценка меньше или равна L1-оценке.

**Какую использовать?**
- Первый раз делаете эксперимент → **L1** (консервативная оценка)
- Много измерений, случайные ошибки → **L2** (статистическая оценка)
- Нужна гарантия безопасности → **L1** (максимальная погрешность)

**В этом уроке** мы будем рассматривать **оба способа** и визуально сравнивать результаты!

---
## 🔧 Импорты и настройка

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import stats
from typing import Callable, Dict, List, Optional, Tuple
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Для вычисления линий уровня
import matplotlib.pyplot as plt

# Настройка для красивого вывода
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4, suppress=True)

print("✅ Библиотеки загружены успешно!")

---
## 🎯 Класс ErrorAnalyzer

Создадим универсальный класс для анализа распространения погрешностей:

**Возможности:**
- Аналитический расчёт (L1 и L2)
- Монте-Карло симуляция (1000 случайных измерений)
- Визуализация 2D с линиями уровня функции
- Поддержка коррелированных ошибок

In [ ]:
class ErrorAnalyzer:
    """
    Класс для анализа распространения погрешностей.
    
    Поддерживает:
    - Аналитический расчёт (L1 и L2 нормы)
    - Монте-Карло симуляцию
    - Визуализацию 2D с линиями уровня
    - Коррелированные ошибки
    """
    
    def __init__(self, 
                 func: Callable,
                 var_names: List[str],
                 nominal_values: Dict[str, float],
                 errors: Dict[str, float],
                 result_name: str = 'z',
                 result_unit: str = ''):
        """
        Parameters:
        -----------
        func : callable
            Функция для анализа, например: lambda m, V: m / V
        var_names : list of str
            Названия переменных (в том же порядке, что и аргументы func)
            Например: ['m', 'V']
        nominal_values : dict
            Номинальные значения, например: {'m': 150, 'V': 50}
        errors : dict
            Погрешности (стандартные отклонения), например: {'m': 1, 'V': 2}
        result_name : str
            Название результата (для графиков)
        result_unit : str
            Единицы измерения результата
        """
        self.func = func
        self.var_names = var_names
        self.nominal_values = nominal_values
        self.errors = errors
        self.result_name = result_name
        self.result_unit = result_unit
        
        # Вычисляем номинальное значение результата
        args = [nominal_values[v] for v in var_names]
        self.nominal_result = func(*args)
        
        # Данные Монте-Карло (будут заполнены при вызове monte_carlo)
        self.mc_inputs = None
        self.mc_outputs = None
    
    def compute_analytical(self, derivatives: Dict[str, Callable]) -> Dict:
        """
        Аналитический расчёт погрешностей L1 и L2.
        
        Parameters:
        -----------
        derivatives : dict
            Частные производные, например:
            {'m': lambda m, V: 1/V, 'V': lambda m, V: -m/V**2}
        
        Returns:
        --------
        dict: {'L1': float, 'L2': float, 'result': float}
        """
        args = [self.nominal_values[v] for v in self.var_names]
        
        # Вычисляем вклады от каждой переменной
        contributions_L1 = []
        contributions_L2_sq = []
        
        for var in self.var_names:
            if var in derivatives:
                deriv_value = derivatives[var](*args)
                error = self.errors[var]
                
                contrib = abs(deriv_value) * error
                contributions_L1.append(contrib)
                contributions_L2_sq.append((deriv_value * error) ** 2)
        
        # L1: линейное сложение
        delta_L1 = sum(contributions_L1)
        
        # L2: квадратичное сложение
        delta_L2 = np.sqrt(sum(contributions_L2_sq))
        
        return {
            'result': self.nominal_result,
            'L1': delta_L1,
            'L2': delta_L2
        }
    
    def monte_carlo(self, 
                    n_samples: int = 1000,
                    correlated: Optional[Dict] = None,
                    seed: int = 42) -> Dict:
        """
        Монте-Карло симуляция распространения погрешностей.
        
        Parameters:
        -----------
        n_samples : int
            Количество случайных точек
        correlated : dict, optional
            Информация о корреляциях, например:
            {
                'type': 'scale_error',  # тип корреляции
                'vars': ['D', 'h'],     # какие переменные коррелируют
                'scale_std': 0.02       # σ систематической ошибки масштаба
            }
        seed : int
            Для воспроизводимости результатов
        
        Returns:
        --------
        dict: {'inputs': DataFrame, 'outputs': array, 'std': float, 'mean': float}
        """
        np.random.seed(seed)
        
        # Генерируем случайные значения для каждой переменной
        samples = {}
        
        # Сначала генерируем независимые случайные ошибки
        for var in self.var_names:
            nominal = self.nominal_values[var]
            error = self.errors[var]
            # Нормальное распределение с центром в номинальном значении
            samples[var] = np.random.normal(nominal, error, n_samples)
        
        # Если есть корреляция - добавляем систематическую ошибку
        if correlated is not None:
            if correlated['type'] == 'scale_error':
                # Ошибка масштаба: все коррелированные переменные 
                # умножаются на (1 + ε), где ε одинаковое для всех
                scale_std = correlated['scale_std']
                corr_vars = correlated['vars']
                
                # Генерируем одну систематическую ошибку для всех измерений
                epsilon = np.random.normal(0, scale_std, n_samples)
                
                # Применяем к коррелированным переменным
                for var in corr_vars:
                    if var in samples:
                        samples[var] = samples[var] * (1 + epsilon)
        
        # Создаём DataFrame для удобства
        self.mc_inputs = pd.DataFrame(samples)
        
        # Вычисляем результаты для каждого набора измерений
        outputs = []
        for i in range(n_samples):
            args = [samples[var][i] for var in self.var_names]
            outputs.append(self.func(*args))
        
        self.mc_outputs = np.array(outputs)
        
        return {
            'inputs': self.mc_inputs,
            'outputs': self.mc_outputs,
            'mean': np.mean(self.mc_outputs),
            'std': np.std(self.mc_outputs, ddof=1)  # выборочное стандартное отклонение
        }
    
    def visualize_2d(self, 
                     show_ellipses: bool = True,
                     analytical_result: Optional[Dict] = None,
                     title: str = '') -> go.Figure:
        """
        Визуализация на плоскости (для функций 2 переменных).
        
        Показывает:
        - Облако точек из Монте-Карло симуляции
        - Номинальную точку
        - Эллипс 70% доверительной области (χ², характеристика распределения входных данных)
        - Линии уровня функции (изолинии): правда, правда±δL1, правда±δL2
        
        Parameters:
        -----------
        show_ellipses : bool
            Параметр сохранён для обратной совместимости (не используется)
        analytical_result : dict, optional
            Результаты аналитического расчёта (из compute_analytical)
        title : str
            Заголовок графика
        
        Returns:
        --------
        plotly Figure
        """
        if self.mc_inputs is None:
            raise ValueError("Сначала запустите monte_carlo()!")
        
        if len(self.var_names) != 2:
            raise ValueError("Для 2D визуализации нужно ровно 2 переменные!")
        
        var_x, var_y = self.var_names
        
        # Создаём фигуру
        fig = go.Figure()
        
        # 1. Облако точек из Монте-Карло
        n_mc = len(self.mc_outputs)
        fig.add_trace(go.Scatter(
            x=self.mc_inputs[var_x],
            y=self.mc_inputs[var_y],
            mode='markers',
            marker=dict(
                size=3,
                color='blue',
                opacity=0.3
            ),
            name=f'Монте-Карло (N={n_mc})',
            hovertemplate=f'{var_x}: %{{x:.3f}}<br>{var_y}: %{{y:.3f}}<extra></extra>'
        ))
        
        # 2. Номинальная точка
        x_nom = self.nominal_values[var_x]
        y_nom = self.nominal_values[var_y]
        fig.add_trace(go.Scatter(
            x=[x_nom],
            y=[y_nom],
            mode='markers',
            marker=dict(
                size=12,
                color='black',
                symbol='x',
                line=dict(width=2)
            ),
            name=f'Номинальное ({var_x}={x_nom:.3g}, {var_y}={y_nom:.3g})',
            hovertemplate=f'{var_x}: %{{x:.3f}}<br>{var_y}: %{{y:.3f}}<extra></extra>'
        ))
        
        # 3. Эллипс 70% доверительной области (χ², 2 степени свободы)
        chi2_70 = stats.chi2.ppf(0.70, df=2)  # ≈ 2.41
        x_data = self.mc_inputs[var_x].values
        y_data = self.mc_inputs[var_y].values
        cov = np.cov(x_data, y_data)
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues, eigenvectors = eigenvalues[idx], eigenvectors[:, idx]
        center_x, center_y = np.mean(x_data), np.mean(y_data)
        theta = np.linspace(0, 2*np.pi, 100)
        unit_circle = np.array([np.cos(theta), np.sin(theta)])
        coord = np.sqrt(chi2_70) * (eigenvectors @ np.diag(np.sqrt(eigenvalues)) @ unit_circle)
        ellipse_x = center_x + coord[0, :]
        ellipse_y = center_y + coord[1, :]
        fig.add_trace(go.Scatter(
            x=ellipse_x, y=ellipse_y,
            mode='lines',
            line=dict(color='orange', width=2, dash='dot'),
            name='Эллипс 70% (χ²)',
            fill='toself', fillcolor='rgba(255,165,0,0.03)',
            hoverinfo='skip'
        ))
        
        # 4. Линии x, y для правды и погрешностей
        dx = self.errors[var_x]
        dy = self.errors[var_y]
        x_lo, x_hi = x_nom - 4*dx, x_nom + 4*dx
        y_lo, y_hi = y_nom - 4*dy, y_nom + 4*dy
        # Вертикальные: x = const
        for x_val, name, style, show in [
            (x_nom, f'{var_x}={x_nom:.3g} (правда)', 'solid', True),
            (x_nom + dx, f'{var_x}={x_nom:.3g}±{dx:.3g}', 'dot', True),
            (x_nom - dx, None, 'dot', False),
        ]:
            fig.add_trace(go.Scatter(x=[x_val, x_val], y=[y_lo, y_hi], mode='lines',
                line=dict(color='gray', width=1, dash=style), name=name or '', showlegend=show, hoverinfo='skip'))
        # Горизонтальные: y = const
        for y_val, name, style, show in [
            (y_nom, f'{var_y}={y_nom:.3g} (правда)', 'solid', True),
            (y_nom + dy, f'{var_y}={y_nom:.3g}±{dy:.3g}', 'dot', True),
            (y_nom - dy, None, 'dot', False),
        ]:
            fig.add_trace(go.Scatter(x=[x_lo, x_hi], y=[y_val, y_val], mode='lines',
                line=dict(color='gray', width=1, dash=style), name=name or '', showlegend=show, hoverinfo='skip'))
        
        # 5. Линии уровня (если есть аналитический результат)
        if analytical_result is not None:
            # Создаём сетку вокруг номинальных значений
            # Увеличиваем область, чтобы линии уровня были полностью видны
            x_range = np.linspace(x_nom - 4*dx, x_nom + 4*dx, 300)
            y_range = np.linspace(y_nom - 4*dy, y_nom + 4*dy, 300)
            X, Y = np.meshgrid(x_range, y_range)
            
            # Вычисляем значения функции на сетке
            Z = np.zeros_like(X)
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    args = [X[i, j], Y[i, j]]
                    Z[i, j] = self.func(*args)
            
            # Определяем уровни и их параметры отображения
            # (уровень, цвет, название, стиль линии, толщина)
            unit_str = f' {self.result_unit}' if self.result_unit else ''
            z0 = self.nominal_result
            zn = self.result_name
            levels_data = [
                (z0, 'black', f'{zn}={z0:.4g}{unit_str} (правда)', 'solid', 3),
                (z0 + analytical_result['L1'], 'red', f'{zn}={z0 + analytical_result["L1"]:.4g}{unit_str} (+δL1)', 'dash', 2),
                (z0 - analytical_result['L1'], 'red', f'{zn}={z0 - analytical_result["L1"]:.4g}{unit_str} (-δL1)', 'dash', 2),
                (z0 + analytical_result['L2'], 'green', f'{zn}={z0 + analytical_result["L2"]:.4g}{unit_str} (+δL2)', 'solid', 2),
                (z0 - analytical_result['L2'], 'green', f'{zn}={z0 - analytical_result["L2"]:.4g}{unit_str} (-δL2)', 'solid', 2),
            ]
            
            # Строим контуры для каждого уровня
            for level_value, color, name, dash, width in levels_data:
                # Используем matplotlib для вычисления контура
                cs = plt.contour(X, Y, Z, levels=[level_value])
                
                # Извлекаем пути контуров (matplotlib 3.8+: get_paths() вместо .collections)
                for path_idx, path in enumerate(cs.get_paths()):
                    vertices = path.vertices
                    
                    # Показываем в легенде только первый контур для каждого уровня
                    show_in_legend = (path_idx == 0)
                    
                    fig.add_trace(go.Scatter(
                        x=vertices[:, 0],
                        y=vertices[:, 1],
                        mode='lines',
                        line=dict(color=color, width=width, dash=dash),
                        name=name,
                        showlegend=show_in_legend,
                        hoverinfo='skip'
                    ))
                
                # Закрываем matplotlib фигуру, чтобы не отображалась
                plt.close()
        
        # Настройка осей и оформления
        fig.update_layout(
            title=title if title else f'Распространение погрешностей: {self.result_name} = f({var_x}, {var_y})',
            xaxis_title=var_x,
            yaxis_title=var_y,
            width=1200,
            height=800,
            hovermode='closest',
            showlegend=True,
            template='plotly_white'
        )
        
        return fig
    
    def summary(self, analytical_result: Optional[Dict] = None, mc_result: Optional[Dict] = None):
        """
        Печать сводки результатов.
        
        Parameters:
        -----------
        analytical_result : dict, optional
            Результаты из compute_analytical()
        mc_result : dict, optional
            Результаты из monte_carlo()
        """
        print("=" * 60)
        print(f"📊 РЕЗУЛЬТАТЫ АНАЛИЗА ПОГРЕШНОСТЕЙ")
        print("=" * 60)
        
        # Номинальное значение
        print(f"\n🎯 Номинальное значение: {self.result_name} = {self.nominal_result:.4f} {self.result_unit}")
        
        # Аналитический расчёт
        if analytical_result:
            print(f"\n📐 Аналитический расчёт:")
            print(f"   L1-норма: δ{self.result_name} = ± {analytical_result['L1']:.4f} {self.result_unit}")
            print(f"   L2-норма: δ{self.result_name} = ± {analytical_result['L2']:.4f} {self.result_unit}")
            print(f"   ")
            print(f"   Результат (L1): {self.result_name} = {self.nominal_result:.4f} ± {analytical_result['L1']:.4f} {self.result_unit}")
            print(f"   Результат (L2): {self.result_name} = {self.nominal_result:.4f} ± {analytical_result['L2']:.4f} {self.result_unit}")
            
            # Относительные погрешности
            rel_L1 = (analytical_result['L1'] / abs(self.nominal_result)) * 100
            rel_L2 = (analytical_result['L2'] / abs(self.nominal_result)) * 100
            print(f"\n   Относительная погрешность (L1): {rel_L1:.2f}%")
            print(f"   Относительная погрешность (L2): {rel_L2:.2f}%")
        
        # Монте-Карло
        if mc_result:
            print(f"\n🎲 Монте-Карло симуляция (N={len(self.mc_outputs)}):")
            print(f"   Среднее: {mc_result['mean']:.4f} {self.result_unit}")
            print(f"   Стандартное отклонение: ± {mc_result['std']:.4f} {self.result_unit}")
            print(f"   ")
            print(f"   Результат (MC): {self.result_name} = {mc_result['mean']:.4f} ± {mc_result['std']:.4f} {self.result_unit}")
            
            rel_mc = (mc_result['std'] / abs(mc_result['mean'])) * 100
            print(f"   Относительная погрешность (MC): {rel_mc:.2f}%")
        
        # Сравнение и распределение точек по диапазонам
        if analytical_result and mc_result:
            print(f"\n🔍 Сравнение методов:")
            print(f"   L1 / MC = {analytical_result['L1'] / mc_result['std']:.3f}")
            print(f"   L2 / MC = {analytical_result['L2'] / mc_result['std']:.3f}")
            
            # Процент точек в каждом диапазоне погрешности
            n_total = len(self.mc_outputs)
            in_L2 = np.sum(np.abs(self.mc_outputs - self.nominal_result) <= analytical_result['L2'])
            in_L1 = np.sum(np.abs(self.mc_outputs - self.nominal_result) <= analytical_result['L1'])
            out_L1 = n_total - in_L1
            
            print(f"\n📈 Распределение точек Монте-Карло по диапазонам:")
            print(f"   В пределах ±δL2: {in_L2} / {n_total} = {100 * in_L2 / n_total:.1f}%")
            print(f"   В пределах ±δL1: {in_L1} / {n_total} = {100 * in_L1 / n_total:.1f}%")
            print(f"   Вне ±δL1:       {out_L1} / {n_total} = {100 * out_L1 / n_total:.1f}%")
            print(f"   ")
            print(f"   ✅ L2-норма ближе к Монте-Карло (т.к. обе учитывают статистику)")
            print(f"   ✅ L1-норма даёт консервативную (завышенную) оценку")
        
        print("\n" + "=" * 60)

---
## 🎨 Демонстрация на простой функции

Для начала попробуем простую функцию, чтобы понять, как работает класс.

**Функция:** $z = x \cdot y$ (произведение)

**Дано:**
- $x = 10 \pm 1$
- $y = 5 \pm 0.5$

**Частные производные:**
- $\frac{\partial z}{\partial x} = y$
- $\frac{\partial z}{\partial y} = x$

In [ ]:
# Создаём анализатор
demo = ErrorAnalyzer(
    func=lambda x, y: x * y,
    var_names=['x', 'y'],
    nominal_values={'x': 10, 'y': 5},
    errors={'x': 1, 'y': 0.5},
    result_name='z',
    result_unit=''
)

# Аналитический расчёт
analytical = demo.compute_analytical({
    'x': lambda x, y: y,  # ∂z/∂x = y
    'y': lambda x, y: x   # ∂z/∂y = x
})

# Монте-Карло
mc = demo.monte_carlo(n_samples=1000)

# Сводка результатов
demo.summary(analytical, mc)

In [ ]:
# Визуализация
fig = demo.visualize_2d(
    analytical_result=analytical,  # Передаём аналитические результаты для линий уровня
    title='Демонстрация: z = x · y'
)
fig.show()

**Что мы видим на графике:**

1. 🔵 **Синие точки** — результаты 1000 случайных "измерений" из Монте-Карло
2. ✖️ **Чёрный крестик** — номинальные значения (x=10, y=5)
3. ⬛ **Чёрная жирная линия** — изолиния "правда" (все точки где z = 50)
4. 🔴 **Красные пунктирные линии** — изолинии "правда ± δL1" (консервативная оценка)
5. 🟢 **Зелёные сплошные линии** — изолинии "правда ± δL2" (статистическая оценка)

**Наблюдение:**
- Синие точки образуют эллиптическое облако (нормальное распределение)
- Линии уровня показывают, какие комбинации (x, y) дают одинаковое значение z
- Зелёные линии (L2) ближе к чёрной, чем красные (L1) — L2-оценка меньше
- Область между линиями "правда ± δL2" содержит большинство синих точек

---
## 📝 Задача 1: Плотность ρ = m/V

**Формула:** $\rho = \frac{m}{V}$

**Физический смысл:** Измеряем массу и объём вещества, вычисляем плотность.

**Задание:** Выведите частные производные $\frac{\partial \rho}{\partial m}$ и $\frac{\partial \rho}{\partial V}$ используя правило дифференцирования частного.

**Дано:**
- Масса: $m = 150$ г $\pm 1$ г (взвешивание на весах)
- Объём: $V = 50$ мл $\pm 2$ мл (мерный стакан)

In [ ]:
# Задача 1: Плотность
task1 = ErrorAnalyzer(
    func=lambda m, V: m / V,
    var_names=['m', 'V'],
    nominal_values={'m': 150, 'V': 50},
    errors={'m': 1, 'V': 2},
    result_name='ρ',
    result_unit='г/мл'
)

# TODO: вычислить аналитически
# task1_analytical = task1.compute_analytical({
#     'm': lambda m, V: ...,
#     'V': lambda m, V: ...
# })

# TODO: Монте-Карло
# task1_mc = task1.monte_carlo(n_samples=1000)

# TODO: Результаты
# task1.summary(task1_analytical, task1_mc)

---
## 📝 Задача 2: Кинетическая энергия E = ½mv²

**Формула:** $E = \frac{1}{2}mv^2$

**Задание:** Выведите частные производные $\frac{\partial E}{\partial m}$ и $\frac{\partial E}{\partial v}$.

**Дано:**
- Масса: $m = 2$ кг $\pm 0.1$ кг
- Скорость: $v = 10$ м/с $\pm 0.5$ м/с

In [ ]:
# Задача 2: Кинетическая энергия
task2 = ErrorAnalyzer(
    func=lambda m, v: 0.5 * m * v**2,
    var_names=['m', 'v'],
    nominal_values={'m': 2, 'v': 10},
    errors={'m': 0.1, 'v': 0.5},
    result_name='E',
    result_unit='Дж'
)

# TODO: вычислить аналитически
# task2_analytical = task2.compute_analytical({...})

# TODO: Монте-Карло
# task2_mc = task2.monte_carlo(n_samples=1000)

# TODO: Результаты
# task2.summary(task2_analytical, task2_mc)

---
## 📝 Задача 3: Период маятника T = 2π√(L/g)

**Формула:** $T = 2\pi\sqrt{\frac{L}{g}}$

**Задание:** Выведите частные производные $\frac{\partial T}{\partial L}$ и $\frac{\partial T}{\partial g}$ используя цепное правило для степенных функций.

**Подсказка:** Перепишите формулу как $T = 2\pi \cdot L^{1/2} \cdot g^{-1/2}$

**Дано:**
- Длина: $L = 1.00$ м $\pm 0.01$ м
- Ускорение свободного падения: $g = 9.81$ м/с² $\pm 0.05$ м/с²

In [ ]:
# Задача 3: Период маятника
task3 = ErrorAnalyzer(
    func=lambda L, g: 2 * np.pi * np.sqrt(L / g),
    var_names=['L', 'g'],
    nominal_values={'L': 1.0, 'g': 9.81},
    errors={'L': 0.01, 'g': 0.05},
    result_name='T',
    result_unit='с'
)

# TODO: вычислить аналитически
# task3_analytical = task3.compute_analytical({...})

# TODO: Монте-Карло
# task3_mc = task3.monte_carlo(n_samples=1000)

# TODO: Результаты
# task3.summary(task3_analytical, task3_mc)

---
## 📝 Задача 4: Радиоактивный распад N = N₀·e^(-λt)

**Формула:** $N = N_0 \cdot e^{-\lambda t}$

**Задание:** Выведите частные производные $\frac{\partial N}{\partial N_0}$ и $\frac{\partial N}{\partial t}$.

**Подсказка:** Используйте правило дифференцирования экспоненты и цепное правило.

**Дано:**
- Начальное количество: $N_0 = 1000 \pm 10$ частиц
- Константа распада: $\lambda = 0.1$ с$^{-1}$ (известно точно)
- Время: $t = 5$ с $\pm 0.2$ с

**Примечание:** λ известно точно, поэтому анализируем только (N₀, t).

In [ ]:
# Задача 4: Радиоактивный распад
# λ = 0.1 (константа, без погрешности)
lambda_decay = 0.1

task4 = ErrorAnalyzer(
    func=lambda N0, t: N0 * np.exp(-lambda_decay * t),
    var_names=['N0', 't'],
    nominal_values={'N0': 1000, 't': 5},
    errors={'N0': 10, 't': 0.2},
    result_name='N',
    result_unit='частиц'
)

# TODO: вычислить аналитически
# task4_analytical = task4.compute_analytical({...})

# TODO: Монте-Карло
# task4_mc = task4.monte_carlo(n_samples=1000)

# TODO: Результаты
# task4.summary(task4_analytical, task4_mc)

---
## 📝 Задача 5: Угол наклонной плоскости α = arctg(h/L)

**Формула:** $\alpha = \arctg\left(\frac{h}{L}\right)$

**Задание:** Выведите частные производные $\frac{\partial\alpha}{\partial h}$ и $\frac{\partial\alpha}{\partial L}$.

**Подсказка:** Используйте производную обратной функции арктангенса: $(\arctg(u))' = \frac{1}{1+u^2} \cdot u'$ и цепное правило.

**Дано:**
- Высота: $h = 0.5$ м $\pm 0.01$ м
- Длина основания: $L = 2.0$ м $\pm 0.02$ м

In [ ]:
# Задача 5: Угол наклонной плоскости
task5 = ErrorAnalyzer(
    func=lambda h, L: np.arctan(h / L),
    var_names=['h', 'L'],
    nominal_values={'h': 0.5, 'L': 2.0},
    errors={'h': 0.01, 'L': 0.02},
    result_name='α',
    result_unit='рад'
)

# TODO: вычислить аналитически
# task5_analytical = task5.compute_analytical({...})

# TODO: Монте-Карло
# task5_mc = task5.monte_carlo(n_samples=1000)

# TODO: Результаты
# task5.summary(task5_analytical, task5_mc)

---
## 🎯 Задача 6 (БОНУС): Объем цилиндра с коррелированными ошибками

### Мотивация: Почему ошибки могут быть коррелированными?

До сих пор мы предполагали, что погрешности **независимы**: ошибка в измерении $x$ никак не связана с ошибкой в измерении $y$.

Но в реальных экспериментах это не всегда так!

**Пример:** Измеряем диаметр $D$ и высоту $h$ цилиндра **одной и той же линейкой**.

- Если линейка имеет **систематическую ошибку масштаба** (например, растянулась от температуры или неправильно откалибрована)
- То оба измерения ($D$ и $h$) будут ошибочны **в одну сторону**
- Погрешности **коррелируют**!

---

### Формула объёма цилиндра

$$V = \pi r^2 h = \frac{\pi}{4} D^2 h$$

где $D$ — диаметр, $h$ — высота.

**Частные производные:**
$$\frac{\partial V}{\partial D} = \frac{\pi}{2} D h$$

$$\frac{\partial V}{\partial h} = \frac{\pi}{4} D^2$$

---

### Модель ошибок

**Две компоненты погрешности:**

1. **Случайная ошибка считывания** (независимая для каждого измерения):
   - $\delta D_{\text{случ}}$ ~ $N(0, \sigma_D)$
   - $\delta h_{\text{случ}}$ ~ $N(0, \sigma_h)$

2. **Систематическая ошибка масштаба линейки** (общая для обоих измерений):
   - Линейка показывает на $\varepsilon\%$ больше/меньше реального
   - $\varepsilon$ ~ $N(0, \sigma_{\text{масштаб}})$
   - $D_{\text{измер}} = D_{\text{истин}} \cdot (1 + \varepsilon)$
   - $h_{\text{измер}} = h_{\text{истин}} \cdot (1 + \varepsilon)$

**Итого:**
$$D_{\text{измер}} = D_{\text{истин}} \cdot (1 + \varepsilon) + \delta D_{\text{случ}}$$
$$h_{\text{измер}} = h_{\text{истин}} \cdot (1 + \varepsilon) + \delta h_{\text{случ}}$$

где $\varepsilon$ — **одно и то же** для обоих измерений!

---

### Что мы увидим?

На графике облако точек будет **наклонено** вдоль диагонали!

Это визуально покажет корреляцию: если $D$ больше номинала, то и $h$ с большей вероятностью тоже больше (из-за общей $\varepsilon$).

**Эллипс ошибок наклонён!**

---

### Дано:
- Диаметр: $D = 5.0$ см $\pm 0.1$ см (случайная ошибка считывания)
- Высота: $h = 10.0$ см $\pm 0.1$ см (случайная ошибка считывания)
- Систематическая ошибка масштаба линейки: $\sigma_{\text{масштаб}} = 2\%$

In [ ]:
# Задача 6: Объем цилиндра (с корреляцией)
task6 = ErrorAnalyzer(
    func=lambda D, h: (np.pi / 4) * D**2 * h,
    var_names=['D', 'h'],
    nominal_values={'D': 5.0, 'h': 10.0},
    errors={'D': 0.1, 'h': 0.1},  # случайные ошибки считывания
    result_name='V',
    result_unit='см³'
)

# Аналитический расчёт (БЕЗ учёта корреляции)
task6_analytical = task6.compute_analytical({
    'D': lambda D, h: (np.pi / 2) * D * h,      # ∂V/∂D = (π/2)·D·h
    'h': lambda D, h: (np.pi / 4) * D**2        # ∂V/∂h = (π/4)·D²
})

print("="*60)
print("📊 СРАВНЕНИЕ: С корреляцией vs БЕЗ корреляции")
print("="*60)

# Монте-Карло БЕЗ корреляции
print("\n1️⃣ Без корреляции (только случайные ошибки считывания):")
task6_mc_uncorrelated = task6.monte_carlo(n_samples=1000, correlated=None)
print(f"   Стандартное отклонение: ± {task6_mc_uncorrelated['std']:.4f} см³")

# Монте-Карло С корреляцией (добавляем систематическую ошибку масштаба)
print("\n2️⃣ С корреляцией (+ систематическая ошибка масштаба 2%):")
task6_mc_correlated = task6.monte_carlo(
    n_samples=1000,
    correlated={
        'type': 'scale_error',
        'vars': ['D', 'h'],
        'scale_std': 0.02  # σ_масштаб = 2%
    },
    seed=123  # другой seed для разных результатов
)
print(f"   Стандартное отклонение: ± {task6_mc_correlated['std']:.4f} см³")

print("\n📈 Вывод:")
print(f"   Корреляция УВЕЛИЧИВАЕТ погрешность в {task6_mc_correlated['std'] / task6_mc_uncorrelated['std']:.2f} раз!")
print(f"   Это происходит потому, что систематическая ошибка влияет на ОБА измерения.")
print("\n" + "="*60)

In [ ]:
# Визуализация БЕЗ корреляции
# (восстанавливаем данные БЕЗ корреляции в task6)
task6.mc_inputs = task6_mc_uncorrelated['inputs']
task6.mc_outputs = task6_mc_uncorrelated['outputs']

fig6a = task6.visualize_2d(
    analytical_result=task6_analytical,  # Передаём аналитические результаты
    title='Задача 6a: Объем цилиндра БЕЗ корреляции'
)
fig6a.show()

In [ ]:
# Визуализация С корреляцией
task6.mc_inputs = task6_mc_correlated['inputs']
task6.mc_outputs = task6_mc_correlated['outputs']

fig6b = task6.visualize_2d(
    analytical_result=task6_analytical,  # Передаём аналитические результаты
    title='Задача 6b: Объем цилиндра С КОРРЕЛЯЦИЕЙ (наклонный эллипс!)'
)
fig6b.show()

### 🔍 Анализ результатов задачи 6

**Что мы видим на графиках:**

**График 6a (БЕЗ корреляции):**
- Облако точек симметрично относительно осей
- Эллипс ошибок **не наклонён**
- Ошибки в $D$ и $h$ независимы

**График 6b (С КОРРЕЛЯЦИЕЙ):**
- Облако точек **наклонено вдоль диагонали**
- Эллипс ошибок **наклонён**! 🎯
- Если $D$ больше номинала, то и $h$ с большей вероятностью тоже больше
- Это визуализация **коррелированных ошибок**

**Физический смысл:**
- Систематическая ошибка линейки влияет на ОБА измерения одновременно
- Погрешность результата **больше**, чем при независимых ошибках
- В реальных экспериментах важно учитывать такие корреляции!

---

### 💡 Важный вывод

Формулы L1 и L2 **предполагают независимость ошибок**.

Если ошибки коррелируют:
- Нужно использовать более сложные методы (ковариационная матрица)
- Или разделить источники ошибок: использовать **разные измерительные приборы** для $D$ и $h$!

---
## 🎓 Заключение

В этом ноутбуке мы:

1. ✅ Изучили две нормы для распространения погрешностей: **L1** (консервативная) и **L2** (статистическая)
2. ✅ Реализовали универсальный класс `ErrorAnalyzer` для анализа
3. ✅ Решили 5 физических задач с визуализацией:
   - Плотность (правило частного)
   - Кинетическая энергия (произведение и степень)
   - Период маятника (цепное правило)
   - Радиоактивный распад (экспонента)
   - Угол наклонной плоскости (обратная функция)
4. ✅ Изучили **коррелированные ошибки** на примере объёма цилиндра
5. ✅ Увидели **наклонный эллипс** — визуализацию корреляции!

**Ключевые навыки:**
- Применять частные производные для анализа погрешностей
- Различать L1 и L2 подходы
- Использовать Монте-Карло для проверки аналитических расчётов
- Визуализировать распространение ошибок
- Понимать влияние коррелированных ошибок

---

### 📚 Домашнее задание

1. Модифицируйте задачу 2 (кинетическая энергия): добавьте третью переменную — высоту $h$. Вычислите полную механическую энергию $E_{\text{полн}} = \frac{1}{2}mv^2 + mgh$.

2. Придумайте свою физическую задачу с 2 переменными и решите её с помощью `ErrorAnalyzer`.

3. Исследуйте, как изменяется погрешность объёма цилиндра (задача 6) при увеличении систематической ошибки масштаба с 1% до 5%. Постройте график зависимости.

4. **(Сложное)** Реализуйте метод для 3D визуализации функций трёх переменных.